In [16]:
import json
import datetime as dt

In [ ]:
# load expenses.json
with open("local_dev/expenses.json", "r") as f:
    expenses = json.load(f)

expenses['months'].sort(key=lambda x: dt.datetime.strptime(x['month'], '%B %Y'))

In [17]:
dt.datetime.strptime('April 2023', '%B %Y')

datetime.datetime(2023, 4, 1, 0, 0)

In [1]:
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
import os
from dotenv import dotenv_values
from functools import partial
from sqlalchemy import text

secrets = dotenv_values(".env")


def create_connection_url(server, database, username, password, port, driver):
    connection_url = URL.create(
        "mssql+pyodbc",
        username=username,
        password=password,
        host=server,
        port=port,
        database=database,
        query={
            "driver": driver,
            "TrustServerCertificate": "No",
        }
    )
    return connection_url


def test_connection(connection_string):
    try:
        connection_string = connection_string
        engine = create_engine(connection_string)
        with engine.connect() as connection:  # get a connection
            print("Successfully connected to the database.")
            result = connection.execute(text(
                'SELECT @@VERSION'))  # execute the query
            row = result.fetchone()  # fetch the result
            print(f"SQL Server version: {row[0]}")
            assert row[0] is not None
    except Exception as e:
        # If the connection fails, the test will fail with an exception
        assert False, f'Connection test failed: {e}'


server = secrets['azure_sql_server']
database = secrets['azure_sql_database']
username = secrets['azure_DB_username']
password = secrets['azure_DB_password']
port = secrets['port']
driver = secrets['azure_DB_driver']

CreateURL = partial(create_connection_url, server=server,
                    username=username, password=password, port=port, driver=driver,)

In [2]:
test_connection(CreateURL(database=database))

Successfully connected to the database.
SQL Server version: Microsoft SQL Azure (RTM) - 12.0.2000.8 
	Jul  8 2023 12:00:47 
	Copyright (C) 2022 Microsoft Corporation

